In [6]:
from qiskit_braket_provider import AWSBraketProvider
from qiskit import *
import numpy as np
from qiskit.providers.fake_provider import FakeMumbaiV2
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import QuantumVolume
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
    resource_id="/subscriptions/6648b6bc-19eb-4d18-9855-d527b2cc7f29/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iontrap",
    location='East US'
)
#provider.get_backend('IonQ Device') 
QuantumVolume(2).decompose().decompose().decompose().draw(output='mpl')
#np.random.seed(8122)

def get_noise(p_meas,p_gate, p_gate2):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(depolarizing_error(p_gate2, 1))

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

SHOTS=1024
num_qubits = 11
control_qubit = 0

ion_chain=QuantumRegister(num_qubits)
readout=ClassicalRegister(2)
qc=QuantumCircuit(ion_chain, readout)
errors=[]
backend=provider.get_backend('ionq.qpu')
#backend=Aer.get_backend('qasm_simulator')
init_vector = np.zeros(2**num_qubits)
init_vector[0] = 1

from qiskit import transpile
run=False
i=1
if run is True:
    #qc.initialize([1,0,0,0], [control_qubit, i])
    qc.x(control_qubit)
    qc.cx(control_qubit,i)
    qc.measure_all()
    circ=transpile(qc, backend, optimization_level=0) 
    counts=backend.run(circ, shots=SHOTS).result().get_counts()
    #counts=backend.run(circ, noise_model=get_noise(0.0018,0.004, 0.027)).result().get_counts()
    errors.append(counts)

def get_error_rates(job):
    return [1-(errors[i]['11']/SHOTS) for i in range(len(job))]


qc.initialize([1,0,0,0], [control_qubit, x])
qc.x(control_qubit)
qc.cx(control_qubit,x)
qc.measure([ion_chain[control_qubit], ion_chain[x]], [readout[0], readout[x+1]])
circ=transpile(qc, backend)
counts=backend.run(circ, shots=SHOTS).result().get_counts()
#counts=backend.run(circ, noise_model=get_noise(0.0018,0.004, 0.027)).result().get_counts()
errors.append(counts)

In [1]:
from azure.quantum import Workspace
from qiskit.visualization import plot_histogram

job_dict={
    '10': 'c9a2f130-c9e4-11ed-81e3-7085c295e290',
    '9' : 'efd403d4-c9e4-11ed-8463-7085c295e290',
    '8' : 'fab1e7f9-c9e4-11ed-9e0c-7085c295e290',
    '7' : '04139531-c9e5-11ed-81f4-7085c295e290',
    '6' : '0d62b484-c9e5-11ed-80c7-7085c295e290',
    '5' : '163c6620-c9e5-11ed-abad-7085c295e290',
    '4' : '1e1b5ec6-c9e5-11ed-935b-7085c295e290',
    '3' : '290fd4c6-c9e5-11ed-a5d8-7085c295e290',
    '2' : '3181439b-c9e5-11ed-b5b9-7085c295e290',
    '1' : '6689d669-c9e5-11ed-8ce5-7085c295e290'}

workspace=Workspace(
    resource_id="/subscriptions/6648b6bc-19eb-4d18-9855-d527b2cc7f29/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iontrap",
    location='East US'
)

def get_error_rates(job):
    job=job['histogram']
    binary=[bin(int(val))[2:].rjust(11, '0') for val in job.keys()]
    job_new={}
    for key, prob in zip(binary, job.values()):
        job_new[key] = prob
    return job_new

raw=[]
for id in job_dict.values():
    job=workspace.get_job(id).get_results()
    job=get_error_rates(job)
    print(job)
    raw.append(job)
raw.reverse()


{'00000000000': 0.001953125, '00000000001': 0.03125, '00001000001': 0.0009765625, '10000000000': 0.005859375, '10000000001': 0.9423828125, '10000000011': 0.0009765625, '10000000101': 0.0009765625, '10000001001': 0.0009765625, '10000100001': 0.0029296875, '10001000001': 0.001953125, '10010000001': 0.0029296875, '11000000001': 0.0068359375}
{'00000000000': 0.0048828125, '00000000001': 0.0126953125, '01000000000': 0.00390625, '01000000001': 0.966796875, '01000000101': 0.0029296875, '01000100001': 0.0009765625, '01001000001': 0.0009765625, '01010000001': 0.0048828125, '01100000001': 0.001953125}
{'00000000000': 0.001953125, '00000000001': 0.0146484375, '00100000000': 0.0029296875, '00100000001': 0.9658203125, '00100000011': 0.0009765625, '00100000101': 0.0029296875, '00100001001': 0.0009765625, '00100010001': 0.001953125, '00100100001': 0.0009765625, '00101000001': 0.00390625, '00110000001': 0.0029296875}
{'00000000000': 0.0048828125, '00000000001': 0.0302734375, '00000000011': 0.000976562

In [32]:
heavy = [max(key.values()) for key in raw]
heavy    

[0.9873046875,
 0.9521484375,
 0.9716796875,
 0.962890625,
 0.9638671875,
 0.931640625,
 0.94921875,
 0.9658203125,
 0.966796875,
 0.9423828125]

In [9]:
import numpy as np
from qiskit_braket_provider import providers
from braket.circuits import Circuit
from braket.aws import AwsDevice
from math import pi
import qiskit_experiments.library.randomized_benchmarking as rb
import qiskit

nQ=2
rb_opts={}
rb['']
rb_opts['length_vector'] = [1, 10, 20, 50, 75, 100]
rb_opts['seeds'] = 7

noise_model = NoiseModel()

# Depolarizing error on the gates u2, u3 and cx (assuming the u1 is virtual-Z gate and no error)
p1Q = 0.002
p2Q = 0.01

noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2 * p1Q, 1), 'u3')
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2Q, 2), 'cx')
rb_circs, xdata = rb.StandardRB()

backend = qiskit.Aer.get_backend('qasm_simulator')
qregs = rb_circs[0][-1].qregs
cregs = rb_circs[0][-1].cregs
qc = qiskit.QuantumCircuit(*qregs, *cregs)
for i in rb_circs[0][-1][0:-nQ]:
    qc.data.append(i)

# Create the RB fitter
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] 
shots = 200
transpiled_circs_list = []
rb_fit = rb.RBFitter(None, xdata, rb_opts['rb_pattern'])
for rb_seed, rb_circ_seed in enumerate(rb_circs):
    print('Compiling seed %d'%rb_seed)
    new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
    transpiled_circs_list.append(new_rb_circ_seed)
    print('Simulating seed %d'%rb_seed)
    job = qiskit.execute(new_rb_circ_seed, backend, shots=shots,
                         noise_model=noise_model,
                         backend_options={'max_parallel_experiments': 0})    
    # Add data to the fitter
    rb_fit.add_data(job.result())
    print('After seed %d, alpha: %f, EPC: %f'%(rb_seed,rb_fit.fit[0]['params'][1], rb_fit.fit[0]['epc']))

TypeError: cannot unpack non-iterable module object

In [29]:
import numpy as np
from qiskit_braket_provider import providers
from braket.circuits import Circuit
from braket.aws import AwsDevice
from math import pi
import qiskit_experiments.library.randomized_benchmarking as rb
import qiskit 
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error
vector_length=[1,10,25,50]
# Run on a noisy simulator
noise_model = NoiseModel()

# Depolarizing error on the gates u2, u3 and cx (assuming the u1 is virtual-Z gate and no error)
p1Q = 0.002
p2Q = 0.01

noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2 * p1Q, 1), 'u3')
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2Q, 2), 'cx')

backend = qiskit.Aer.get_backend('qasm_simulator')

for length in vector_length:
    circ=rb.StandardRB([0,2], vector_length,num_samples=2).circuits()
    circ=qiskit.transpile(circ, backend, optimization_level=0)
    
    job=backend.run(circ, shots=200, noise_model=noise_model).result().get_counts()
    job_dict[i][length] = job
job_dict




{0: {1: [{'00': 198, '10': 1, '01': 1},
   {'00': 182, '10': 7, '01': 9, '11': 2},
   {'00': 159, '11': 16, '01': 11, '10': 14},
   {'10': 32, '00': 121, '11': 34, '01': 13},
   {'00': 192, '01': 5, '11': 2, '10': 1},
   {'00': 171, '11': 8, '10': 13, '01': 8},
   {'00': 155, '01': 14, '10': 15, '11': 16},
   {'00': 124, '11': 23, '01': 30, '10': 23}],
  10: [{'00': 195, '11': 2, '10': 2, '01': 1},
   {'10': 16, '00': 164, '11': 11, '01': 9},
   {'00': 151, '01': 14, '11': 17, '10': 18},
   {'00': 115, '10': 29, '01': 30, '11': 26},
   {'00': 196, '10': 1, '11': 2, '01': 1},
   {'00': 175, '11': 8, '01': 10, '10': 7},
   {'00': 154, '10': 14, '01': 18, '11': 14},
   {'00': 136, '10': 21, '01': 21, '11': 22}],
  25: [{'00': 197, '10': 2, '01': 1},
   {'00': 181, '10': 7, '01': 3, '11': 9},
   {'00': 151, '10': 17, '11': 14, '01': 18},
   {'00': 114, '10': 32, '01': 30, '11': 24},
   {'00': 195, '11': 2, '01': 2, '10': 1},
   {'00': 178, '10': 9, '01': 4, '11': 9},
   {'00': 153, '01': 1

In [19]:
circ[0].draw()

global phase: 3π/2
         ░                                                          »
   q_0: ─░───────────────────X─────────────────────────────■────────»
         ░      ┌───┐        │                 ┌───┐     ┌─┴─┐      »
   q_1: ─░──────┤ H ├────────X────■────■────■──┤ X ├─────┤ X ├──────»
         ░ ┌────┴───┴─────┐┌───┐  │    │    │  └─┬─┘     └───┘      »
   q_2: ─░─┤ U2(π/2,-π/2) ├┤ X ├──┼────┼────┼────┼──────────────────»
         ░ └┬────────────┬┘└─┬─┘  │    │    │    │  ┌──────────────┐»
   q_3: ─░──┤ U2(0,-π/2) ├───┼────┼────┼────┼────■──┤ U2(π/2,-π/2) ├»
         ░  └───┬───┬────┘   │  ┌─┴─┐  │    │       └──────────────┘»
   q_4: ─░──────┤ S ├────────┼──┤ X ├──┼────┼───────────────────────»
         ░      └───┘        │  └───┘  │    │                       »
   q_5: ─░───────────────────■─────────┼────┼───────────────────────»
         ░                           ┌─┴─┐  │                       »
   q_6: ─░───────────────────────────┤ X ├──┼───────────────────────»
         ░      ┌───┐                └───┘┌─┴─┐                     »
   q_7: ─░──────┤ S ├─────────────────────┤ X ├─────────────────────»
         ░      └───┘                     └───┘                     »
meas: 8/════════════════════════════════════════════════════════════»
                                                                    »
«        ┌──────────────┐                               ┌───┐              »
«   q_0: ┤ U2(π/2,-π/2) ├──■─────────────────────────■──┤ X ├──────────────»
«        └────┬───┬─────┘  │                         │  └─┬─┘              »
«   q_1: ─────┤ X ├────────┼────■────────────────────┼────┼────────────────»
«             └─┬─┘        │    │   ┌────────────┐   │    │  ┌────────────┐»
«   q_2: ───────■──────────┼────┼───┤ U2(0,-π/2) ├───┼────■──┤ U2(0,-π/2) ├»
«                          │    │   └────────────┘   │  ┌───┐└────────────┘»
«   q_3: ──────────────────┼────┼────────────────────┼──┤ X ├──────────────»
«                        ┌─┴─┐  │       ┌───┐        │  └─┬─┘              »
«   q_4: ────────────────┤ X ├──┼───────┤ H ├────────┼────┼────────────────»
«                        └───┘┌─┴─┐┌────┴───┴─────┐  │    │                »
«   q_5: ─────────────────────┤ X ├┤ U2(π/2,-π/2) ├──┼────■────────────────»
«                             └───┘└──────────────┘┌─┴─┐┌───┐              »
«   q_6: ──────────────────────────────────────────┤ X ├┤ H ├──────────────»
«                                                  └───┘└───┘              »
«   q_7: ──────────────────────────────────────────────────────────────────»
«                                                                          »
«meas: 8/══════════════════════════════════════════════════════════════════»
«                                                                          »
«        ┌───┐     ┌───┐                                                       »
«   q_0: ┤ X ├──■──┤ Z ├───────────────────────────────────────────────────────»
«        └─┬─┘  │  └───┘                                                       »
«   q_1: ──┼────┼──────────────────────────────────────────────────────────────»
«          │    │            ┌───┐                     ┌───┐┌───┐              »
«   q_2: ──┼────┼────■───────┤ X ├───────■─────────────┤ X ├┤ X ├──────────────»
«          │    │    │  ┌───┐└─┬─┘       │             └─┬─┘└─┬─┘┌────────────┐»
«   q_3: ──■────┼────┼──┤ X ├──┼─────────┼───────────────┼────■──┤ U2(0,-π/2) ├»
«               │    │  └─┬─┘  │       ┌─┴─┐      ┌───┐  │  ┌───┐└───┬───┬────┘»
«   q_4: ───────┼────┼────┼────┼───────┤ X ├──────┤ H ├──■──┤ S ├────┤ X ├─────»
«             ┌─┴─┐  │    │    │  ┌────┴───┴─────┐└───┘     └───┘    └─┬─┘     »
«   q_5: ─────┤ X ├──┼────┼────■──┤ U2(π/2,-π/2) ├─────────────────────■───────»
«             └───┘┌─┴─┐  │  ┌───┐└──────────────┘                             »
«   q_6: ──────────┤ X ├──┼──┤ H ├─────────────────────────────────────────────»
«                  └───┘  │  └───┘                    

In [16]:

from azure.quantum.qiskit import AzureQuantumProvider

provider = AzureQuantumProvider(
            resource_id = "/subscriptions/6648b6bc-19eb-4d18-9855-d527b2cc7f29/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iontrap",
            location = "eastus")
backend = provider.get_backend("ionq.qpu")
cost = backend.estimate_cost(circ, shots=200)
print(f"Estimated cost: {cost.estimated_total} {cost.currency_code}")

AttributeError: 'list' object has no attribute 'clbits'